In [1]:
import pandas as pd

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df

,Lugar de residencia,Medida,Valor
0,Huelva,Población,510743
1,Huelva,Confirmados PDIA,15988
2,Huelva,Confirmados PDIA 14 días,3791
3,Huelva,Tasa PDIA 14 días,"742,2519740848137"
4,Huelva,Confirmados PDIA 7 días,2212
...,...,...,...
627,Municipio de Huelva sin especificar,Tasa PDIA 14 días,-
628,Municipio de Huelva sin especificar,Confirmados PDIA 7 días,34
629,Municipio de Huelva sin especificar,Total Confirmados,247
630,Municipio de Huelva sin especificar,Curados,116


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  15988.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  4416.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 276 personas en los últimos 7 días 

Un positivo PCR cada 153 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,15988.0,2212.0,3791.0,510743.0,433.094531,742.251974,462.0
Huelva-Costa,9519.0,1522.0,2534.0,288115.0,528.261285,879.509918,351.0
Huelva (capital),4416.0,520.0,938.0,143663.0,361.958194,652.916896,121.0
Isla Cristina,1240.0,407.0,647.0,21264.0,1914.033108,3042.701279,74.0
Condado-Campiña,4391.0,384.0,739.0,155057.0,247.650864,476.598928,67.0
Sierra de Huelva-Andévalo Central,1832.0,272.0,429.0,67571.0,402.539551,634.887748,44.0
Ayamonte,574.0,145.0,203.0,20946.0,692.256278,969.158789,36.0
Lepe,1262.0,146.0,234.0,27431.0,532.244541,853.049470,33.0
Aljaraque,651.0,80.0,172.0,21260.0,376.293509,809.031044,25.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Encinasola,104.0,45.0,93.0,1305.0,3448.275862,7126.436782,11.0
Villalba del Alcor,281.0,76.0,167.0,3338.0,2276.812463,5002.995806,4.0
Villanueva de las Cruces,15.0,8.0,15.0,373.0,2144.772118,4021.447721,7.0
Isla Cristina,1240.0,407.0,647.0,21264.0,1914.033108,3042.701279,74.0
Calañas,121.0,53.0,71.0,2791.0,1898.960946,2543.891078,3.0
Berrocal,6.0,6.0,6.0,312.0,1923.076923,1923.076923,1.0
Cerro de Andévalo (El),99.0,29.0,40.0,2364.0,1226.734349,1692.047377,6.0
Palma del Condado (La),703.0,52.0,118.0,10761.0,483.226466,1096.552365,7.0
Paymogo,24.0,12.0,12.0,1159.0,1035.375324,1035.375324,6.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Bonares,94.0,5.0,22.0,6058.0,82.535490,363.156157,0.0,0.227273
Aroche,78.0,3.0,13.0,3073.0,97.624471,423.039375,0.0,0.230769
Cortegana,167.0,10.0,34.0,4666.0,214.316331,728.675525,0.0,0.294118
Trigueros,146.0,4.0,12.0,7713.0,51.860495,155.581486,1.0,0.333333
Paterna del Campo,73.0,1.0,3.0,3509.0,28.498148,85.494443,0.0,0.333333
Jabugo,49.0,1.0,3.0,2250.0,44.444444,133.333333,0.0,0.333333
Rociana del Condado,152.0,9.0,23.0,7855.0,114.576703,292.807129,3.0,0.391304
Manzanilla,57.0,2.0,5.0,2135.0,93.676815,234.192037,0.0,0.400000
Villablanca,77.0,3.0,7.0,2848.0,105.337079,245.786517,0.0,0.428571
